## Install and import packages

In [2]:
%pip install --upgrade --quiet sagemaker
%pip install --upgrade --quiet langchain
%pip install --upgrade --quiet openai

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip uninstall --quiet typing_extensions --yes
%pip install --upgrade --quiet typing_extensions

Note: you may need to restart the kernel to use updated packages.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 5.3.3 requires pyqt5<5.16, which is not installed.
spyder 5.3.3 requires pyqtwebengine<5.16, which is not installed.
panel 0.13.1 requires bokeh<2.5.0,>=2.4.0, but you have bokeh 3.2.1 which is incompatible.
spyder 5.3.3 requires ipython<8.0.0,>=7.31.1, but you have ipython 8.14.0 which is incompatible.
spyder 5.3.3 requires pylint<3.0,>=2.5.0, but you have pylint 3.0.0a6 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [4]:
import openai
import pandas as pd
import time
# from dotenv import load_dotenv
from langchain.chat_models import ChatAnyscale, ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage
import os
# from multichoice import Multichoice
from prompt_mgr import PromptMgr

## Read input dataset and create prompt

In [7]:
df = pd.read_json('resources/datasets/val_sentence_pairs.json')
df.head()

,article_id,article_sent,correct_sent,correct_sent_id,incorrect_sent,incorrect_sent_id
0,67c3b0a2da09ce32a83c098da1e0aff034e790d0,the abc have reported that those who receive c...,those who receive centrelink payments made up ...,84-0,the abc have reported that those who receive c...,40-0
1,7c4ae3beb354e0a9f11fa09f8fbb6613a50db5cc,five ambitious clubs are locked in a scramble ...,five ambitious clubs are locked in a scramble ...,44-1,five ambitious clubs are locked in a bid for t...,17-1
2,7e37b3e3623ae0ab6568e0e73584b16fae0cc00e,but it wasn't until last year that the 25 year...,the 25 year old from pennsylvania went on a le...,45-1,the 25 year old from pennsylvania went viral o...,4-1
3,cdbfba47929b459daf474a1a3dee37d2351729ea,seven games involving nimes were investigated ...,seven games involving nimes were investigated ...,93-2,seven games involving nimes were arrested last...,83-2
4,696ac804af5fcab8853c583f5c97e1eda6e2493a,the driver's side of the windscreen immediatel...,driver's side of the windscreen immediately sh...,61-2,driver's side were immediately shatters and fa...,42-2


In [19]:
pm = PromptMgr(src_dir = 'resources/environments/answer_first/prompts')

In [20]:
def query_model(row, sm_model_predictor, prompt_mgr, swap_answers = False):
    if swap_answers: 
        prompt = prompt_mgr.bind('consistent').render(
                    article_sent=row['article_sent'], 
                    option_a=row['incorrect_sent'],
                    option_b=row['correct_sent'])
    else: 
        prompt = prompt_mgr.bind('consistent').render(
                    article_sent=row['article_sent'], 
                    option_a=row['correct_sent'],
                    option_b=row['incorrect_sent'])

    system_prompt = prompt_mgr.bind('system').render()
    
    payload = {
        "inputs": [[
            {"role": "user", "content": f"{system_prompt}\n{prompt}"},
        ]],
        "parameters": {"max_new_tokens": 512, "top_p": 0.9, "temperature": 0.6}
    }
    
    model_predictor = sm_model_predictor
    
    response = predictor.predict(payload, custom_attributes='accept_eula=true')
    
    output = response
    return {'output': output[0]['generation']['content']} 

## Deploy Llama-2 7b, save output and delete SM endpoint

In [21]:
model_id, model_version = "meta-textgeneration-llama-2-7b-f", "*"

In [ ]:
from sagemaker.jumpstart.model import JumpStartModel

model = JumpStartModel(model_id=model_id)
predictor = model.deploy()

In [10]:
# Create an empty list to store the results
results_list = []

# Iterate over each row in the DataFrame
for i in range(len(df)):
    result = query_model(df.loc[i], predictor, pm, swap_answers=True)
    results_list.append(result)

# Create a new DataFrame from the results list
results_df = pd.DataFrame(results_list)

In [11]:
results_df.loc[0]['output']

' Based on the article sentence provided, Summary B is more consistent with the article.\n\nThe article sentence states that "the abc have reported that those who receive centrelink payments made up half of radio rental\'s income last year."\n\nSummary B paraphrases this sentence into a more concise form, "those who receive centrelink payments made up half of radio rental\'s income last year."\n\nBoth Summary A and Summary B provide the same information, but Summary B is more concise and easier to read. Therefore, Summary B is the more consistent summary of the article sentence.'

In [13]:
results_df.to_json('Llama-2-7b-output-swapped.json')

In [14]:
# Create an empty list to store the results
results_list_no_swap = []

# Iterate over each row in the DataFrame
for i in range(len(df)):
    result = query_model(df.loc[i], predictor, pm, swap_answers=False)
    results_list_no_swap.append(result)

# Create a new DataFrame from the results list
results_df_no_swap = pd.DataFrame(results_list_no_swap)

In [15]:
results_df_no_swap.loc[0]['output']

' Based on the article sentence provided, I would say that Summary B is more consistent with the article. The article sentence directly states that "the abc have reported that those who receive centrelink payments made up half of radio rental\'s income last year," which implies that the ABC news organization reported on the income of Radio Rental and found that centrelink payments made up half of it. Therefore, Summary B is more consistent with the article as it directly quotes the ABC news organization.'

In [16]:
results_df_no_swap.to_json('Llama-2-7b-output-no-swap.json')

In [17]:
# Delete the SageMaker endpoint
predictor.delete_model()
predictor.delete_endpoint()

## Deploy Llama-2 13b, save output and delete SM endpoint

In [22]:
model_id, model_version = "meta-textgeneration-llama-2-13b-f", "*"

In [23]:
from sagemaker.jumpstart.model import JumpStartModel

model = JumpStartModel(model_id=model_id)
predictor = model.deploy()

-------------------!

In [24]:
# Create an empty list to store the results
results_13b_list = []

# Iterate over each row in the DataFrame
for i in range(len(df)):
    result = query_model(df.loc[i], predictor, pm, swap_answers=True)
    results_13b_list.append(result)

# Create a new DataFrame from the results list
results_13b_df = pd.DataFrame(results_13b_list)

In [25]:
results_13b_df.loc[0]['output']

' Based on the information provided in the article sentence, the more consistent summary with the article would be:\n\nSummary B: those who receive centrelink payments made up half of radio rental\'s income last year.\n\nExplanation:\n\nThe article sentence states that "those who receive Centrelink payments made up half of radio rental\'s income last year." Therefore, the summary that best reflects this information is Summary B, which states "those who receive Centrelink payments made up half of radio rental\'s income last year."\n\nSummary A, on the other hand, states "the abc have reported that those who receive Centrelink payments made up radio rental\'s income last year." This summary is not as precise and factual as Summary B, as it does not directly quote the information from the article and instead uses indirect language to convey the same information.'

In [26]:
results_13b_df.to_json('Llama-2-13b-output-swapped.json')

In [27]:
# Create an empty list to store the results
results_13b_list_no_swap = []

# Iterate over each row in the DataFrame
for i in range(len(df)):
    result = query_model(df.loc[i], predictor, pm, swap_answers=False)
    results_13b_list_no_swap.append(result)

# Create a new DataFrame from the results list
results_13b_df_no_swap = pd.DataFrame(results_13b_list_no_swap)

In [29]:
results_13b_df_no_swap.loc[0]['output']

' Based on the information provided in the article sentence, the more consistent summary is:\n\nSummary A: those who receive centrelink payments made up half of radio rental\'s income last year.\n\nExplanation: The article sentence states that the abc have reported that those who receive centrelink payments made up half of radio rental\'s income last year. Therefore, the summary should be a direct representation of that information, which is: "those who receive centrelink payments made up half of radio rental\'s income last year." Summary B is not as consistent because it adds the phrase "the abc have reported" which is not directly supported by the article sentence.'

In [30]:
results_13b_df_no_swap.to_json('Llama-2-13b-output-no-swap.json')

In [31]:
# Delete the SageMaker endpoint
predictor.delete_model()
predictor.delete_endpoint()

## Deploy Llama-2 70b, save output and delete SM endpoint

In [33]:
model_id, model_version = "meta-textgeneration-llama-2-70b-f", "*"

In [34]:
from sagemaker.jumpstart.model import JumpStartModel

model = JumpStartModel(model_id=model_id)
predictor = model.deploy()

---------------------------------------!

In [35]:
# Create an empty list to store the results
results_70b_list = []

# Iterate over each row in the DataFrame
for i in range(len(df)):
    result = query_model(df.loc[i], predictor, pm, swap_answers=True)
    results_70b_list.append(result)

# Create a new DataFrame from the results list
results_70b_df = pd.DataFrame(results_70b_list)

In [36]:
results_70b_df.loc[0]['output']

" The correct answer is (B) those who receive centrelink payments made up half of radio rental's income last year.\n\nExplanation:\n\nSummary A is not consistent with the article sentence as it implies that the ABC (Australian Broadcasting Corporation) made up radio rental's income, which is not supported by the article.\n\nSummary B, on the other hand, accurately summarizes the article sentence by stating that those who receive Centrelink payments (a government benefit) made up half of radio rental's income last year, which is consistent with the information provided in the article."

In [37]:
results_70b_df.to_json('Llama-2-70b-output-swapped.json')

In [45]:
# Create an empty list to store the results
results_70b_list_no_swap = []

# Iterate over each row in the DataFrame
for i in range(len(df)):
    result = query_model(df.loc[i], predictor, pm, swap_answers=False)
    results_70b_list_no_swap.append(result)

# Create a new DataFrame from the results list
results_70b_df_no_swap = pd.DataFrame(results_70b_list_no_swap)

In [46]:
results_70b_df_no_swap.loc[0]['output']

' The correct answer is (A) those who receive centrelink payments made up half of radio rental\'s income last year.\n\nExplanation:\n\nThe article sentence states that "the abc have reported" certain information. Summary A accurately reflects this by stating that "those who receive centrelink payments made up half of radio rental\'s income last year," which is a direct reflection of the information provided in the article sentence.\n\nOn the other hand, Summary B, "the abc have reported that those who receive centrelink payments made up radio rental\'s income last year," is not consistent with the article sentence. The article sentence does not state that the ABC reported that centrelink payments made up radio rental\'s entire income, only that those who receive centrelink payments made up half of radio rental\'s income. Therefore, Summary B is not supported by the information provided in the article sentence.'

In [47]:
results_70b_df_no_swap.to_json('Llama-2-70b-output-no-swap.json')

In [48]:
# Delete the SageMaker endpoint
predictor.delete_model()
predictor.delete_endpoint()

## Postprocessing

In [64]:
# deploy llama-2-70b to extract the answer
model_id, model_version = "meta-textgeneration-llama-2-70b-f", "*"

from sagemaker.jumpstart.model import JumpStartModel

model = JumpStartModel(model_id=model_id)
predictor = model.deploy()

---------------------------------------!

In [49]:
input_df = df

In [72]:
llama_2_7b_swapped_df  = pd.read_json('Llama-2-7b-output-swapped.json')
llama_2_7b_no_swap_df  = pd.read_json('Llama-2-7b-output-no-swap.json')

llama_2_13b_swapped_df = pd.read_json('Llama-2-13b-output-swapped.json')
llama_2_13b_no_swap_df = pd.read_json('Llama-2-13b-output-no-swap.json')

llama_2_70b_swapped_df = pd.read_json('Llama-2-70b-output-swapped.json')
llama_2_70b_no_swap_df = pd.read_json('Llama-2-70b-output-no-swap.json')

In [74]:
def extract_answer(row, sm_model_predictor, prompt_mgr):
    multichoice = prompt_mgr.bind('multichoice_query').render(query=row['output'])
    
    payload = {
        "inputs": [[
            {"role": "user", "content": f"{multichoice}"},
        ]],
        "parameters": {"max_new_tokens": 512, "top_p": 0.9, "temperature": 0.6}
    }
    
    model_predictor = sm_model_predictor
    
    response = predictor.predict(payload, custom_attributes='accept_eula=true')
    
    output = response
    
    return {'output': output[0]['generation']['content']} 

In [77]:
# llama_2_7b_swapped_df
temp_df = llama_2_7b_swapped_df

# Create an empty list to store the results
results_list = []

# Iterate over each row in the DataFrame
for i in range(len(df)):
    result = extract_answer(temp_df.loc[i], predictor, pm)
    results_list.append(result)

# Create a new DataFrame from the results list
llama_2_7b_swapped_df_extracted = pd.DataFrame(results_list)

llama_2_7b_swapped_df_extracted.to_json('Llama-2-7b-output-swapped-extracted.json')

In [81]:
# llama_2_7b_no_swap_df
temp_df = llama_2_7b_no_swap_df

# Create an empty list to store the results
results_list = []

# Iterate over each row in the DataFrame
for i in range(len(df)):
    result = extract_answer(temp_df.loc[i], predictor, pm)
    results_list.append(result)

# Create a new DataFrame from the results list
llama_2_7b_no_swap_df_extracted = pd.DataFrame(results_list)

llama_2_7b_no_swap_df_extracted.to_json('Llama-2-7b-output-no-swap-extracted.json')

In [82]:
# llama_2_13b_swapped_df
temp_df = llama_2_13b_swapped_df

# Create an empty list to store the results
results_list = []

# Iterate over each row in the DataFrame
for i in range(len(df)):
    result = extract_answer(temp_df.loc[i], predictor, pm)
    results_list.append(result)

# Create a new DataFrame from the results list
llama_2_13b_swapped_df_extracted = pd.DataFrame(results_list)

llama_2_13b_swapped_df_extracted.to_json('Llama-2-13b-output-swapped-extracted.json')

In [83]:
# llama_2_13b_no_swap_df
temp_df = llama_2_13b_no_swap_df

# Create an empty list to store the results
results_list = []

# Iterate over each row in the DataFrame
for i in range(len(df)):
    result = extract_answer(temp_df.loc[i], predictor, pm)
    results_list.append(result)

# Create a new DataFrame from the results list
llama_2_13b_no_swap_df_extracted = pd.DataFrame(results_list)

llama_2_13b_no_swap_df_extracted.to_json('Llama-2-13b-output-no-swap-extracted.json')

In [84]:
# llama_2_70b_swapped_df
temp_df = llama_2_70b_swapped_df

# Create an empty list to store the results
results_list = []

# Iterate over each row in the DataFrame
for i in range(len(df)):
    result = extract_answer(temp_df.loc[i], predictor, pm)
    results_list.append(result)

# Create a new DataFrame from the results list
llama_2_70b_swapped_df_extracted = pd.DataFrame(results_list)

llama_2_70b_swapped_df_extracted.to_json('Llama-2-70b-output-swapped-extracted.json')

In [85]:
# llama_2_70b_no_swap_df
temp_df = llama_2_70b_no_swap_df

# Create an empty list to store the results
results_list = []

# Iterate over each row in the DataFrame
for i in range(len(df)):
    result = extract_answer(temp_df.loc[i], predictor, pm)
    results_list.append(result)

# Create a new DataFrame from the results list
llama_2_70b_no_swap_df_extracted = pd.DataFrame(results_list)

llama_2_70b_no_swap_df_extracted.to_json('Llama-2-70b-output-no-swap-extracted.json')

In [87]:
# Delete the SageMaker endpoint
predictor.delete_model()
predictor.delete_endpoint()

## Analysis

In [ ]:
# list of dataframes

# llama_2_7b_swapped_df
# llama_2_7b_swapped_df_extracted
# llama_2_7b_no_swap_df
# llama_2_7b_no_swap_df_extracted

# llama_2_13b_swapped_df
# llama_2_13b_swapped_df_extracted
# llama_2_13b_no_swap_df
# llama_2_13b_no_swap_df_extracted

# llama_2_70b_swapped_df
# llama_2_70b_swapped_df_extracted
# llama_2_70b_no_swap_df
# llama_2_70b_no_swap_df_extracted

In [88]:
llama_2_7b_swapped_df.rename(columns={'output': '7b_swapped_output'}, inplace=True)
llama_2_7b_no_swap_df.rename(columns={'output': '7b_no_swap_output'}, inplace=True)

llama_2_13b_swapped_df.rename(columns={'output': '13b_swapped_output'}, inplace=True)
llama_2_13b_no_swap_df.rename(columns={'output': '13b_no_swap_output'}, inplace=True)

llama_2_70b_swapped_df.rename(columns={'output': '70b_swapped_output'}, inplace=True)
llama_2_70b_no_swap_df.rename(columns={'output': '70b_no_swap_output'}, inplace=True)


llama_2_7b_swapped_df_extracted.rename(columns={'output': '7b_swapped_output_extracted'}, inplace=True)
llama_2_7b_no_swap_df_extracted.rename(columns={'output': '7b_no_swap_output_extracted'}, inplace=True)

llama_2_13b_swapped_df_extracted.rename(columns={'output': '13b_swapped_output_extracted'}, inplace=True)
llama_2_13b_no_swap_df_extracted.rename(columns={'output': '13b_no_swap_output_extracted'}, inplace=True)

llama_2_70b_swapped_df_extracted.rename(columns={'output': '70b_swapped_output_extracted'}, inplace=True)
llama_2_70b_no_swap_df_extracted.rename(columns={'output': '70b_no_swap_output_extracted'}, inplace=True)

In [105]:
combined_df = pd.concat([input_df, 
                         llama_2_7b_swapped_df, 
                         llama_2_7b_swapped_df_extracted,
                         llama_2_7b_no_swap_df,
                         llama_2_7b_no_swap_df_extracted,
                         llama_2_13b_swapped_df, 
                         llama_2_13b_swapped_df_extracted,
                         llama_2_13b_no_swap_df,
                         llama_2_13b_no_swap_df_extracted,
                         llama_2_70b_swapped_df, 
                         llama_2_70b_swapped_df_extracted,
                         llama_2_70b_no_swap_df,
                         llama_2_70b_no_swap_df_extracted,], axis=1)

In [106]:
combined_df.head()

,article_id,article_sent,correct_sent,correct_sent_id,incorrect_sent,incorrect_sent_id,7b_swapped_output,7b_swapped_output_extracted,7b_no_swap_output,7b_no_swap_output_extracted,13b_swapped_output,13b_swapped_output_extracted,13b_no_swap_output,13b_no_swap_output_extracted,70b_swapped_output,70b_swapped_output_extracted,70b_no_swap_output,70b_no_swap_output_extracted
0,67c3b0a2da09ce32a83c098da1e0aff034e790d0,the abc have reported that those who receive c...,those who receive centrelink payments made up ...,84-0,the abc have reported that those who receive c...,40-0,"Based on the article sentence provided, Summa...",B,"Based on the article sentence provided, I wou...",B,Based on the information provided in the arti...,B,Based on the information provided in the arti...,A,The correct answer is (B) those who receive c...,B,The correct answer is (A) those who receive c...,A
1,7c4ae3beb354e0a9f11fa09f8fbb6613a50db5cc,five ambitious clubs are locked in a scramble ...,five ambitious clubs are locked in a scramble ...,44-1,five ambitious clubs are locked in a bid for t...,17-1,"Based on the article sentence provided, Summa...",B,"Based on the article sentence provided, I wou...",B,"Based on the article sentence, the more consi...",B,"Based on the article sentence, the more consi...",A,The correct answer is (B) five ambitious club...,B,The correct answer is (A) five ambitious club...,A
2,7e37b3e3623ae0ab6568e0e73584b16fae0cc00e,but it wasn't until last year that the 25 year...,the 25 year old from pennsylvania went on a le...,45-1,the 25 year old from pennsylvania went viral o...,4-1,"Based on the article sentence provided, I wou...",B,Based on the information provided in the arti...,B,Based on the information provided in the arti...,B,Based on the information provided in the arti...,A,The correct answer is (B) the 25 year old fro...,B,The correct answer is A: the 25 year old from...,A
3,cdbfba47929b459daf474a1a3dee37d2351729ea,seven games involving nimes were investigated ...,seven games involving nimes were investigated ...,93-2,seven games involving nimes were arrested last...,83-2,"Based on the article sentence provided, the m...",B,"Based on the article sentence provided, the m...",B,Based on the information provided in the arti...,B,Based on the information provided in the arti...,A,The correct answer is (B) seven games involvi...,B,The correct answer is (A) seven games involvi...,A
4,696ac804af5fcab8853c583f5c97e1eda6e2493a,the driver's side of the windscreen immediatel...,driver's side of the windscreen immediately sh...,61-2,driver's side were immediately shatters and fa...,42-2,"Based on the article sentence, Summary B is m...",B,"Based on the article sentence, Summary B is m...",B,"Based on the article sentence, the more consi...",B,"Based on the article sentence, the more consi...",A,The correct answer is (B) driver's side of th...,B,"The correct answer is A: ""driver's side of th...",A


In [107]:
combined_df.tail()

,article_id,article_sent,correct_sent,correct_sent_id,incorrect_sent,incorrect_sent_id,7b_swapped_output,7b_swapped_output_extracted,7b_no_swap_output,7b_no_swap_output_extracted,13b_swapped_output,13b_swapped_output_extracted,13b_no_swap_output,13b_no_swap_output_extracted,70b_swapped_output,70b_swapped_output_extracted,70b_no_swap_output,70b_no_swap_output_extracted
368,7682ecf708029ba1328e6b5b1f8fe841efa92495,former manchester city boss roberto mancini ha...,roberto mancini has turned up the heat on succ...,82-4,roberto mancini has turned up the heat for man...,76-4,"Based on the article sentence provided, I wou...",B,"Based on the article sentence provided, Summa...",B,Based on the information provided in the arti...,B,Based on the information provided in the arti...,A,The correct answer is (B) Summary B: roberto ...,B,The correct answer is A: roberto mancini has ...,A
369,7c4ae3beb354e0a9f11fa09f8fbb6613a50db5cc,louis van gaal said that he believed wenger's ...,louis van gaal says he believed wenger is esti...,44-2,louis van gaal says wenger's estimation of the...,86-2,"Based on the article sentence, the more consi...",B,"Based on the article sentence, the more consi...",B,"Based on the article sentence, the more consi...",A,Based on the information provided in the arti...,A,The correct answer is A: louis van gaal says ...,A,The correct answer is (B) louis van gaal says...,B
370,06819c41879edbe99b8612b552afd80b96356ef7,lloris wearing the scarf after defeat by chels...,lloris wearing the scarf after defeat by chels...,19-2,lloris wearing the scarf after defeat by chels...,12-2,"Based on the article sentence provided, the m...",B,"Based on the article sentence provided, Summa...",A,Based on the information provided in the arti...,A,Based on the information provided in the arti...,A,A,A,The correct answer is (B) lloris wearing the ...,B
371,85506154c3f291ddd0cc3935dc09215f9c4437ed,in an anonymous confession to australia's gold...,the woman reveals she once had sex with two gu...,22-2,the woman reveals she was supposed to be seein...,60-2,I cannot provide an answer to this question a...,Other,"I apologize, but I cannot provide an answer t...",Other,"Based on the article sentence, the more consi...",B,Based on the information provided in the arti...,A,The correct answer is (B).\n\nThe article sen...,B,The correct answer is (A) The woman reveals s...,A
372,7378d362db4e1308fc139a9a7b7e156fd3547bf1,he was also ordered to pay $ 167 million in re...,he also ordered to pay $ 167 million in restit...,54-2,he was also ordered to pay $ 167 million in da...,49-2,"Based on the article sentence provided, the m...",B,"Based on the article sentence, Summary A is m...",A,Based on the information provided in the arti...,B,"Based on the article sentence, the more consi...",A,The correct answer is (B) he also ordered to ...,B,The correct answer is A: he also ordered to p...,A


In [110]:
import re 

def remove_spaces(text):
    return re.sub(r"\s+", "", text)

In [112]:
combined_df['7b_swapped_output_extracted'] = combined_df['7b_swapped_output_extracted'].apply(remove_spaces)
combined_df['7b_no_swap_output_extracted'] = combined_df['7b_no_swap_output_extracted'].apply(remove_spaces)

combined_df['13b_swapped_output_extracted'] = combined_df['13b_swapped_output_extracted'].apply(remove_spaces)
combined_df['13b_no_swap_output_extracted'] = combined_df['13b_no_swap_output_extracted'].apply(remove_spaces)

combined_df['70b_swapped_output_extracted'] = combined_df['70b_swapped_output_extracted'].apply(remove_spaces)
combined_df['70b_no_swap_output_extracted'] = combined_df['70b_no_swap_output_extracted'].apply(remove_spaces)

In [117]:
def is_correct(row): 
    if (row[0] == 'B' and row[1] == 'A'):
        return 'Y'
    if (row[0] == row[1]):
        return row[0]*2
    return 'N'

In [118]:
combined_df['7b_analysis_result'] = combined_df[['7b_swapped_output_extracted', '7b_no_swap_output_extracted']].apply(is_correct, axis=1)
combined_df['13b_analysis_result'] = combined_df[['13b_swapped_output_extracted', '13b_no_swap_output_extracted']].apply(is_correct, axis=1)
combined_df['70b_analysis_result'] = combined_df[['70b_swapped_output_extracted', '70b_no_swap_output_extracted']].apply(is_correct, axis=1)

In [120]:
combined_df.head()

,article_id,article_sent,correct_sent,correct_sent_id,incorrect_sent,incorrect_sent_id,7b_swapped_output,7b_swapped_output_extracted,7b_no_swap_output,7b_no_swap_output_extracted,...,13b_swapped_output_extracted,13b_no_swap_output,13b_no_swap_output_extracted,70b_swapped_output,70b_swapped_output_extracted,70b_no_swap_output,70b_no_swap_output_extracted,7b_analysis_result,13b_analysis_result,70b_analysis_result
0,67c3b0a2da09ce32a83c098da1e0aff034e790d0,the abc have reported that those who receive c...,those who receive centrelink payments made up ...,84-0,the abc have reported that those who receive c...,40-0,"Based on the article sentence provided, Summa...",B,"Based on the article sentence provided, I wou...",B,...,B,Based on the information provided in the arti...,A,The correct answer is (B) those who receive c...,B,The correct answer is (A) those who receive c...,A,BB,Y,Y
1,7c4ae3beb354e0a9f11fa09f8fbb6613a50db5cc,five ambitious clubs are locked in a scramble ...,five ambitious clubs are locked in a scramble ...,44-1,five ambitious clubs are locked in a bid for t...,17-1,"Based on the article sentence provided, Summa...",B,"Based on the article sentence provided, I wou...",B,...,B,"Based on the article sentence, the more consi...",A,The correct answer is (B) five ambitious club...,B,The correct answer is (A) five ambitious club...,A,BB,Y,Y
2,7e37b3e3623ae0ab6568e0e73584b16fae0cc00e,but it wasn't until last year that the 25 year...,the 25 year old from pennsylvania went on a le...,45-1,the 25 year old from pennsylvania went viral o...,4-1,"Based on the article sentence provided, I wou...",B,Based on the information provided in the arti...,B,...,B,Based on the information provided in the arti...,A,The correct answer is (B) the 25 year old fro...,B,The correct answer is A: the 25 year old from...,A,BB,Y,Y
3,cdbfba47929b459daf474a1a3dee37d2351729ea,seven games involving nimes were investigated ...,seven games involving nimes were investigated ...,93-2,seven games involving nimes were arrested last...,83-2,"Based on the article sentence provided, the m...",B,"Based on the article sentence provided, the m...",B,...,B,Based on the information provided in the arti...,A,The correct answer is (B) seven games involvi...,B,The correct answer is (A) seven games involvi...,A,BB,Y,Y
4,696ac804af5fcab8853c583f5c97e1eda6e2493a,the driver's side of the windscreen immediatel...,driver's side of the windscreen immediately sh...,61-2,driver's side were immediately shatters and fa...,42-2,"Based on the article sentence, Summary B is m...",B,"Based on the article sentence, Summary B is m...",B,...,B,"Based on the article sentence, the more consi...",A,The correct answer is (B) driver's side of th...,B,"The correct answer is A: ""driver's side of th...",A,BB,Y,Y


In [123]:
def acc_bias(col):
    results = combined_df[col].value_counts()/len(combined_df)
    acc = results['Y']
    if 'AA' not in results:
        results['AA'] = 0
    if 'BB' not in results: 
        results['BB'] = 0
        
    bias = abs(results['AA'] - results['BB'])
    aa_ratio = results['AA'] * 100
    bb_ratio = results['BB'] * 100
    if results['AA'] > results['BB']:
        towards = 'A'
    else:
        towards = 'B'
        
    return acc, aa_ratio, bb_ratio, bias, towards

In [125]:
for m in ['7b', '13b', '70b']:
    acc, aa_ratio, bb_ratio, bias, towards = acc_bias(m+'_analysis_result')
    acc = acc*100
    bias = bias*100
    print(f'{m}:   \tAccuracy: {acc:.1f}%  \tAA: {aa_ratio:.1f}%\tBB: {bb_ratio:.1f}%\tBias: {bias:.1f}% towards {towards}')

7b:   	Accuracy: 23.1%  	AA: 1.1%	BB: 64.6%	Bias: 63.5% towards B
13b:   	Accuracy: 56.3%  	AA: 32.4%	BB: 3.2%	Bias: 29.2% towards A
70b:   	Accuracy: 84.5%  	AA: 3.5%	BB: 8.0%	Bias: 4.6% towards B
